In [ ]:
!pip install selenium

## Part 1: Consider the set of GitHub repositories in the provided link: http://aserg-ufmg.github.io/why-we-refactor/#/projects

### Part 1 a: Web scrapping to get 124 github links
Instruction to use selenium: https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os

# Download and Access WebDriver
# In my case, I download at the link: https://googlechromelabs.github.io/chrome-for-testing/#stable

# Specify the path to the downloaded ChromeDriver
driver_path = os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe")

browser = webdriver.Chrome()
cService = webdriver.ChromeService(executable_path=driver_path)
driver = webdriver.Chrome(service=cService)

# Access Website Via Python
driver.get("http://aserg-ufmg.github.io/why-we-refactor/#/projects")

driver.set_page_load_timeout(50)  # wait for the page to load

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12700\1210067507.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.DataFrame(columns=["Project", "Creation Date", "Commits", "Java Files", "Contributors"])  # creates master dataframe

# Extract project rows from the table
data = driver.find_elements(By.CLASS_NAME, "ng-binding")
list_data = enumerate(data)

print(len(list(list_data)))

# Group data into chunks of 5 for each project
rows = []
for i in range(1, len(data), 5):
    project_link = data[i].text
    creation_date = data[i + 1].text
    commits = data[i + 2].text
    java_files = data[i + 3].text
    contributors = data[i + 4].text

    # Create a dictionary for each project
    row = {
        "Project": project_link,
        "Creation Date": creation_date,
        "Commits": commits,
        "Java Files": java_files,
        "Contributors": contributors,
    }

    # Append each row to the list
    rows.append(row)

# Use pd.concat to combine all rows into the DataFrame
df = pd.concat([df, pd.DataFrame(rows)], ignore_index=True)

# Output the DataFrame
print(df)

621
                                               Project Creation Date Commits  \
0    https://github.com/JetBrains/intellij-communit...       9/30/11  162625   
1                 https://github.com/JetBrains/MPS.git       8/15/11   66445   
2    https://github.com/CyanogenMod/android_framewo...       5/13/13   62208   
3       https://github.com/liferay/liferay-plugins.git       9/25/09   33929   
4                   https://github.com/neo4j/neo4j.git      11/12/12   29187   
..                                                 ...           ...     ...   
119               https://github.com/zeromq/jeromq.git        8/1/12     316   
120          https://github.com/bitfireAT/davdroid.git       8/25/13     291   
121           https://github.com/bennidi/mbassador.git      10/23/12     236   
122        https://github.com/novoda/android-demos.git       7/26/09     142   
123               https://github.com/jfinal/jfinal.git       4/25/12     102   

    Java Files Contributors  
0    

In [3]:
# Close the browser once the data is collected
driver.quit()

### Part 1 b: Divide 124 projects into 5 groups (Keep the last group with 24 projects), randomly select 2 projects from each group. Because we aim for the bonus point, we will do mining for all projects. 
- Hung: 1-25
- Mazen: 26-50
- Juuso: 51-75
- Nicolas: 76-100
- Bharu: 101-124

We won't run this function, as it's used for randomly select 10 projects from 5 groups.

In [11]:
import random
from numpy import sort
# List of 124 project names (example list, you can replace it with actual project names)
projects = [f"{i+1}" for i in range(124)]

# Divide projects into 5 groups, with first 4 groups contain 25 projects each
group_size = 25
groups = [projects[i : i + group_size] for i in range(0, 100, group_size)]
# Add the remaining 24 projects as the last group
groups.append(projects[100:])

# Randomly select 2 projects from each group
selected_projects = []
for group in groups:
    selected_projects.extend(random.sample(group, 2))

selected_projects = [int(project) for project in selected_projects] # Convert to int for sorting
selected_projects = sort(selected_projects)

print(selected_projects)
# Output the selected 10 projects
print("Selected projects to work with:")
for project in selected_projects:
    print("Project_"+str(project))

[  1   9  28  48  61  69  89  91 109 117]
Selected projects to work with:
Project_1
Project_9
Project_28
Project_48
Project_61
Project_69
Project_89
Project_91
Project_109
Project_117


Lists of project that you are assigned to do

In [4]:
Start = 1
End = 25
selected_projects = [i for i in range(Start, End+1)]

print(selected_projects)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


Get github links that we generate above

In [5]:
project_links = []

for project in selected_projects:
    project_link = df.loc[project - 1, "Project"]
    project_links.append(project_link)

# Print the project links
for link in project_links:
    print(link)

https://github.com/JetBrains/intellij-community.git
https://github.com/JetBrains/MPS.git
https://github.com/CyanogenMod/android_frameworks_base.git
https://github.com/liferay/liferay-plugins.git
https://github.com/neo4j/neo4j.git
https://github.com/apache/camel.git
https://github.com/gradle/gradle.git
https://github.com/processing/processing.git
https://github.com/elastic/elasticsearch.git
https://github.com/belaban/JGroups.git
https://github.com/osmandapp/Osmand.git
https://github.com/wildfly/wildfly.git
https://github.com/SonarSource/sonarqube.git
https://github.com/VoltDB/voltdb.git
https://github.com/languagetool-org/languagetool.git
https://github.com/grails/grails-core.git
https://github.com/apache/hive.git
https://github.com/fabric8io/fabric8.git
https://github.com/hazelcast/hazelcast.git
https://github.com/apache/cassandra.git
https://github.com/rstudio/rstudio.git
https://github.com/spring-projects/spring-framework.git
https://github.com/droolsjbpm/drools.git
https://github.co

In [5]:
project_links = [
    "https://github.com/JetBrains/intellij-community.git",
    "https://github.com/JetBrains/MPS.git",
    "https://github.com/CyanogenMod/android_frameworks_base.git",
    "https://github.com/liferay/liferay-plugins.git",
    "https://github.com/neo4j/neo4j.git",
    "https://github.com/apache/camel.git",
    "https://github.com/gradle/gradle.git",
    "https://github.com/processing/processing.git",
    "https://github.com/elastic/elasticsearch.git",
    "https://github.com/belaban/JGroups.git",
    "https://github.com/osmandapp/Osmand.git",
    "https://github.com/wildfly/wildfly.git",
    "https://github.com/SonarSource/sonarqube.git",
    "https://github.com/VoltDB/voltdb.git",
    "https://github.com/languagetool-org/languagetool.git",
    "https://github.com/grails/grails-core.git",
    "https://github.com/apache/hive.git",
    "https://github.com/fabric8io/fabric8.git",
    "https://github.com/hazelcast/hazelcast.git",
    "https://github.com/apache/cassandra.git",
    "https://github.com/rstudio/rstudio.git",
    "https://github.com/spring-projects/spring-framework.git",
    "https://github.com/droolsjbpm/drools.git",
    "https://github.com/BroadleafCommerce/BroadleafCommerce.git",
    "https://github.com/eclipse/jetty.project.git",
]

Get repo name for later use

In [6]:
import os

pwd = os.getcwd()

repo_names = []

for link in project_links:
    # Get the clean repo name by splitting the URL and removing ".git"
    repo_suffix = link.split("/")[-1]
    if repo_suffix.endswith(".git"):
        repo_name = repo_suffix[:-4]  # Remove the last 4 characters ('.git')
    else:
        repo_name = repo_suffix
    print(repo_name)
    repo_names.append(repo_name)

print(repo_names)

intellij-community
MPS
android_frameworks_base
liferay-plugins
neo4j
camel
gradle
processing
elasticsearch
JGroups
Osmand
wildfly
sonarqube
voltdb
languagetool
grails-core
hive
fabric8
hazelcast
cassandra
rstudio
spring-framework
drools
BroadleafCommerce
jetty.project
['intellij-community', 'MPS', 'android_frameworks_base', 'liferay-plugins', 'neo4j', 'camel', 'gradle', 'processing', 'elasticsearch', 'JGroups', 'Osmand', 'wildfly', 'sonarqube', 'voltdb', 'languagetool', 'grails-core', 'hive', 'fabric8', 'hazelcast', 'cassandra', 'rstudio', 'spring-framework', 'drools', 'BroadleafCommerce', 'jetty.project']


Clone 25 github repos to local machine.

Note that you might face error when cloning that returned non-zero exit status 128. If so, fix it by run cmd as admin, run "git config --system core.longpaths true"

In [8]:
import os
import subprocess

pwd = os.getcwd()

for link in project_links:
    # Get the clean repo name by splitting the URL and removing ".git"
    repo_suffix = link.split("/")[-1]
    if repo_suffix.endswith(".git"):
        repo_name = repo_suffix[:-4]  # Remove the last 4 characters ('.git')
    else:
        repo_name = repo_suffix
    # Path to the target directory where the repository will be cloned
    repo_path = os.path.join(pwd, "RepoFolder", repo_name)

    # Cloning the project into the current working directory
    # Check if the directory already exists

    print(f"Cloning project {repo_name}")
    # Cloning the project with error handling
    try:
        subprocess.run(["git", "clone", link, repo_path], check=True)
        print(f"Project {repo_name} cloned\n")
    except subprocess.CalledProcessError as e:
        print(f"Error cloning {repo_name}: {e}")
        continue  # Skip to the next repository if cloning fails

Cloning project intellij-community
Error cloning intellij-community: Command '['git', 'clone', 'https://github.com/JetBrains/intellij-community.git', 'd:\\Oulun\\Period 5\\Software development, Maintenance and Operation\\Projects\\RepoFolder\\intellij-community']' returned non-zero exit status 128.
Cloning project MPS
Error cloning MPS: Command '['git', 'clone', 'https://github.com/JetBrains/MPS.git', 'd:\\Oulun\\Period 5\\Software development, Maintenance and Operation\\Projects\\RepoFolder\\MPS']' returned non-zero exit status 128.
Cloning project android_frameworks_base
Error cloning android_frameworks_base: Command '['git', 'clone', 'https://github.com/CyanogenMod/android_frameworks_base.git', 'd:\\Oulun\\Period 5\\Software development, Maintenance and Operation\\Projects\\RepoFolder\\android_frameworks_base']' returned non-zero exit status 128.
Cloning project liferay-plugins
Error cloning liferay-plugins: Command '['git', 'clone', 'https://github.com/liferay/liferay-plugins.git',

Don't run it for now, keep it last 

Delete projects to free memory (you can do it manually at last :D)

In [13]:
import stat
import shutil
from pathlib import Path

def remove_readonly(func, path, _):
    "Clear the readonly bit and reattempt the removal"
    os.chmod(path, stat.S_IWRITE)
    func(path)


repo_path = os.path.join(pwd, "RepoFolder")
print("Repo path: ", repo_path)

# Clean up: Remove the cloned repository
# if os.path.exists(repo_path):
#     shutil.rmtree(repo_path, onerror=remove_readonly)
    
print(f"Projects erased\n")

Repo path:  d:\Oulun\Period 5\Software development, Maintenance and Operation\Projects\RepoFolder
Projects erased



### Part 1 c: Mine the refactoring activity applied in the history of the cloned projects using RefactoringMiner library
You can read more here: https://github.com/tsantalis/RefactoringMiner

#### Step 1: First thing first, download RefactoringMiner library
- Link and instruction to download: https://github.com/tsantalis/RefactoringMiner/releases
- I use version 3.0.8: https://github.com/tsantalis/RefactoringMiner/releases/download/3.0.8/RefactoringMiner-3.0.8.zip


#### Step 2: Add path of bin to system environment variables
Search google if you don't know how to add

#### Step 3: Make sure it run normally
Make sure you have Java installed on your computer. If you don't, download it here: https://www.oracle.com/java/technologies/javase/jdk17-archive-downloads.html

Then try to run this command line:
> git clone https://github.com/danilofes/refactoring-toy-example.git refactoring-toy-example

> RefactoringMiner -c refactoring-toy-example 36287f7c3b09eff78395267a3ac0d7da067863fd -json result.json

Since release 3.0.0, RefactoringMiner requires Java 17 or newer and Gradle 7.4 or newer

> RefactoringMiner diff --url https://github.com/JabRef/jabref/pull/11180

To run the command above, you must provide a valid OAuth token in the github-oauth.properties file stored in the bin folder. You can generate an OAuth token in GitHub Settings -> Developer settings -> Personal access tokens. Together with creating a file named .github in your home directory (C:\Users\ADMIN\.github for Windows).

The file should contain your GitHub credentials (personal access token or username/password). The format should look like: 

> login=your_github_username

> oauth=your_github_personal_access_token

#### Step 4: Run RefactoringMiner on the cloned repositories

In [7]:
# Now we are ready to look at data dimension to be mined
import subprocess
import os

# repo_paths = [os.path.join(os.getcwd(), "RepoFolder", i) for i in repo_names]
repo_paths = [os.path.join("RepoFolder", i) for i in repo_names]

print(repo_paths)

['RepoFolder\\intellij-community', 'RepoFolder\\MPS', 'RepoFolder\\android_frameworks_base', 'RepoFolder\\liferay-plugins', 'RepoFolder\\neo4j', 'RepoFolder\\camel', 'RepoFolder\\gradle', 'RepoFolder\\processing', 'RepoFolder\\elasticsearch', 'RepoFolder\\JGroups', 'RepoFolder\\Osmand', 'RepoFolder\\wildfly', 'RepoFolder\\sonarqube', 'RepoFolder\\voltdb', 'RepoFolder\\languagetool', 'RepoFolder\\grails-core', 'RepoFolder\\hive', 'RepoFolder\\fabric8', 'RepoFolder\\hazelcast', 'RepoFolder\\cassandra', 'RepoFolder\\rstudio', 'RepoFolder\\spring-framework', 'RepoFolder\\drools', 'RepoFolder\\BroadleafCommerce', 'RepoFolder\\jetty.project']


In [3]:
repo_paths = [
    "RepoFolder\\intellij-community",
    "RepoFolder\\MPS",
    "RepoFolder\\android_frameworks_base",
    "RepoFolder\\liferay-plugins",
    "RepoFolder\\neo4j",
    "RepoFolder\\camel",
    "RepoFolder\\gradle",
    "RepoFolder\\processing",
    "RepoFolder\\elasticsearch",
    "RepoFolder\\JGroups",
    "RepoFolder\\Osmand",
    "RepoFolder\\wildfly",
    "RepoFolder\\sonarqube",
    "RepoFolder\\voltdb",
    "RepoFolder\\languagetool",
    "RepoFolder\\grails-core",
    "RepoFolder\\hive",
    "RepoFolder\\fabric8",
    "RepoFolder\\hazelcast",
    "RepoFolder\\cassandra",
    "RepoFolder\\rstudio",
    "RepoFolder\\spring-framework",
    "RepoFolder\\drools",
    "RepoFolder\\BroadleafCommerce",
    "RepoFolder\\jetty.project",
]
repo_names = [
    "intellij-community",
    "MPS",
    "android_frameworks_base",
    "liferay-plugins",
    "neo4j",
    "camel",
    "gradle",
    "processing",
    "elasticsearch",
    "JGroups",
    "Osmand",
    "wildfly",
    "sonarqube",
    "voltdb",
    "languagetool",
    "grails-core",
    "hive",
    "fabric8",
    "hazelcast",
    "cassandra",
    "rstudio",
    "spring-framework",
    "drools",
    "BroadleafCommerce",
    "jetty.project",
]

In [8]:
import os
from concurrent.futures import ThreadPoolExecutor

# Create the directory for saving results if it doesn't already exist
os.makedirs("RefactoringMiner-Result", exist_ok=True)


def run_command(command, repo_path, save_dir):
    print(f"Starting mining for {repo_path}...")
    try:
        subprocess.run(command, check=True, shell=True)
        print(f"Refactoring mined for {repo_path} saved in {save_dir}")
    except subprocess.CalledProcessError as e:
        print(f"Error mining refactoring for {repo_path}: {e}")


# Run analysis in parallel using ThreadPoolExecutor
with ThreadPoolExecutor(
    max_workers=5
) as executor:  # Adjust the number of workers as needed
    for idx, repo_path in enumerate(repo_paths):
        if idx in [0, 1, 2, 4, 9]:  # Skipping some repos if needed
            continue
        # print(f"Handling for repo number {idx+1}")
        save_dir = os.path.join("RefactoringMiner-Result", f"{repo_names[idx]}.json")
        command = ["RefactoringMiner", "-a", repo_path, "-json", save_dir]
        executor.submit(run_command, command, repo_path, save_dir)

print("Finished mining all data")
# RefactoringMiner -a RepoFolder/neo4j -json RefactoringMiner-Result/neo4j.json
# RefactoringMiner -a RepoFolder/liferay-plugins -json RefactoringMiner-Result/liferay-plugins.json heap space
# RefactoringMiner -a RepoFolder/elasticsearch -json RefactoringMiner-Result/elasticsearch.json heap space
# RefactoringMiner -a RepoFolder/jetty.project -json RefactoringMiner-Result/jetty.project.json
# RefactoringMiner -a RepoFolder/MPS -json RefactoringMiner-Result/MPS.json

Handling for repo number 4
Handling for repo number 6
Handling for repo number 7
Handling for repo number 8
Handling for repo number 9
Handling for repo number 10
Handling for repo number 11
Handling for repo number 12
Handling for repo number 13
Handling for repo number 14
Handling for repo number 15
Handling for repo number 16
Handling for repo number 17
Handling for repo number 18
Handling for repo number 19
Handling for repo number 20
Handling for repo number 21
Handling for repo number 22
Handling for repo number 23
Handling for repo number 24
Handling for repo number 25
Error mining refactoring for RepoFolder\liferay-plugins: Command '['RefactoringMiner', '-a', 'RepoFolder\\liferay-plugins', '-json', 'RefactoringMiner-Result\\liferay-plugins.json']' returned non-zero exit status 1.
Error mining refactoring for RepoFolder\elasticsearch: Command '['RefactoringMiner', '-a', 'RepoFolder\\elasticsearch', '-json', 'RefactoringMiner-Result\\elasticsearch.json']' returned non-zero exit s

### Part 1 d: Collect the commit message from the commits where refactoring activity has been detected

In [2]:
import os
import json
from pydriller import Repository

# Function to get commit message using pydriller
def get_commit_message_from_url(url, repo_name):
    repo_path = os.path.join("RepoFolder", repo_name)  # Local repo path
    commit_hash = url.split('/')[-1]  # Extract commit hash

    try:
        # Use PyDriller to get commit message
        for commit in Repository(repo_path, single=commit_hash).traverse_commits():
            return commit.msg
    except Exception as e:
        print(f"Error retrieving commit message for {commit_hash}: {e}")
        return None


# Function to process refactoring JSON file and extract commit messages
def process_refactoring_json(file_path, output_file, repo_name):
    with open(file_path, "r") as f:
        data = json.load(f)

    data = data.get("commits")
    print(f"Number of commits in {repo_name}: ", len(data))
    # count = 0
    for commit in data:
        commit_id = commit.get("sha1")
        url = commit.get("url")
        refactoring = commit.get("refactorings")

        if not refactoring:
            continue  # Skip if no refactoring is detected
        # count += 1
        # print(count)
        commit_message = get_commit_message_from_url(url, repo_name)

        if commit_message:
            result = {"commit_hash": commit_id, "commit_message": commit_message}

            # Update JSON file incrementally
            with open(output_file, "a") as out_f:
                out_f.write(
                    json.dumps(result, indent=4) + ",\n"
                )  # Append result to file


# Main logic
results_dir = "RefactoringMiner-Result"
output_dir = "CommitMessages"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List of repository names
repo_names = [
    # "neo4j",  # Add all repository names
    "grails-core",
    # ...
]

# Process each repository
for repo_name in repo_names:
    json_file_path = os.path.join(results_dir, f"{repo_name}_final.json")
    output_file = os.path.join(output_dir, f"{repo_name}_commit_messages.json")

    # Clear output file and initialize it
    with open(output_file, "w") as out_f:
        out_f.write("[\n")  # Start JSON array

    if os.path.exists(json_file_path):
        print(f"Processing refactorings for {repo_name}...")
        process_refactoring_json(json_file_path, output_file, repo_name)
        print(f"Commit messages for {repo_name} saved to {output_file}")

    else:
        print(f"Skipping {repo_name} as refactoring results not found")

    # Finalize JSON array at the end of each repository
    with open(output_file, "a") as out_f:
        out_f.write("]\n")  # Close JSON array

print("Finished processing all repositories.")

Processing refactorings for grails-core...
Number of commits in grails-core:  19384
Commit messages for grails-core saved to CommitMessages\grails-core_commit_messages.json
Finished processing all repositories.


### Part 1 e: Calculate and collect the diff change between the detected commit and the previous commit. (Hint: You might use pydriller library if you are using Python for this project.)

In [2]:
import os
import json
from pydriller import Repository


# Function to get commit message, diff, and stats using pydriller
def get_commit_data_from_url(url, repo_name):
    repo_path = os.path.join("RepoFolder", repo_name)  # Local repo path
    commit_hash = url.split("/")[-1]  # Extract commit hash

    try:
        # Use PyDriller to get commit message, diff, and stats
        for commit in Repository(repo_path, single=commit_hash).traverse_commits():
            parent_hash = (
                commit.parents[0] if commit.parents else None
            )  # Get previous commit hash

            # Get diff between the current and previous commit, and calculate stats
            diff_data = []
            diff_stats = {"total_lines_added": 0, "total_lines_deleted": 0}

            for modified_file in commit.modified_files:
                # Collect diff and file path info
                diff = modified_file.diff  # Full diff content for each file
                old_path = modified_file.old_path  # File path before modification
                new_path = (
                    modified_file.new_path
                )  # File path after modification (if renamed)

                # Collect numerical stats: lines added, lines deleted
                lines_added = modified_file.added_lines
                lines_deleted = modified_file.deleted_lines

                # Update overall stats
                diff_stats["total_lines_added"] += lines_added
                diff_stats["total_lines_deleted"] += lines_deleted

                # Append diff info for each modified file
                diff_data.append(
                    {
                        "file": {"old_path": old_path, "new_path": new_path},
                        "diff": diff, 
                        "lines_added": lines_added,
                        "lines_deleted": lines_deleted,
                    }
                )

            # Return commit data including the message, previous commit hash, diff, and stats
            return {
                "commit_hash": commit.hash,
                "previous_commit_hash": parent_hash,
                "commit_message": commit.msg,
                "diff_stats": diff_stats,
                "diff_content": diff_data,
            }

    except Exception as e:
        print(f"Error retrieving commit data for {commit_hash}: {e}")
        return None


# Function to process refactoring JSON file and extract commit messages and diffs
def process_refactoring_json(file_path, output_file, repo_name):
    with open(file_path, "r") as f:
        data = json.load(f)

    data = data.get("commits")
    print(f"Number of commits in {repo_name}: ", len(data))

    for commit in data:
        url = commit.get("url")
        refactoring = commit.get("refactorings")

        if not refactoring:
            continue  # Skip if no refactoring is detected

        # Get commit message, diff, and stats
        commit_data = get_commit_data_from_url(url, repo_name)

        if commit_data:
            # Write the commit message and diff to the output JSON file incrementally
            with open(output_file, "a") as out_f:
                out_f.write(json.dumps(commit_data, indent=4) + ",\n")


# Main logic
results_dir = "RefactoringMiner-Result"
output_dir = "CommitDiff"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List of repository names
repo_names = [
    # Add all repository names here
    "grails-core",
]

# Process each repository
for repo_name in repo_names:
    json_file_path = os.path.join(results_dir, f"{repo_name}_final.json")
    output_file = os.path.join(output_dir, f"{repo_name}_commit_diff.json")

    # Clear output file and initialize it
    with open(output_file, "w") as out_f:
        out_f.write("[\n")  # Start JSON array

    if os.path.exists(json_file_path):
        print(f"Processing refactorings for {repo_name}...")
        process_refactoring_json(json_file_path, output_file, repo_name)
        print(f"Commit messages and diffs for {repo_name} saved to {output_file}")

    else:
        print(f"Skipping {repo_name} as refactoring results not found")

    # Finalize JSON array at the end of each repository
    with open(output_file, "a") as out_f:
        out_f.write("]\n")  # Close JSON array

print("Finished processing all repositories.")

Processing refactorings for grails-core...
Number of commits in grails-core:  19384
Commit messages and diffs for grails-core saved to CommitDiff\grails-core_commit_diff.json
Finished processing all repositories.


## Part 2: SOFTWARE METRICS CALCULATION: Consider the software metrics (product & process) provided in the table at the end of the document.

#### Step 1: Install required libraries
CK: For calculating class-level metrics like CBO, LCOM, and WMC
> pip install pydriller pandas matplotlib seaborn ck

In [ ]:
# import os
# import json
# import pandas as pd
# from pydriller import Repository
# import matplotlib.pyplot as plt
# import seaborn as sns
# from git import Repo  # For extracting author-level metrics


# # Commit-Level Metric Calculations
# def calculate_commit_metrics(commit):
#     """
#     Calculate commit-level metrics from the PyDriller commit object.
#     Metrics:
#         - COMM: The number of commits made to a file up to the considered commit.
#         - ADD: The number of lines added in the commit.
#         - DEL: The number of lines deleted in the commit.
#     """
#     additions = sum(mod.added_lines for mod in commit.modified_files)
#     deletions = sum(mod.deleted_lines for mod in commit.modified_files)
#     num_modified_files = len(commit.modified_files)

#     return {
#         "commit_hash": commit.hash,
#         "author": commit.author.name,
#         "date": commit.committer_date,
#         "additions": additions,  # ADD: Lines added in the commit
#         "deletions": deletions,  # DEL: Lines deleted in the commit
#         "num_files": num_modified_files,  # Number of files modified in the commit
#     }


# # Author-Level Metrics
# def calculate_author_metrics(repo_path):
#     """
#     Calculate author-level metrics using GitPython.
#     Metrics:
#         - ADEV: Number of developers who contributed to a file.
#         - NADEV: Number of distinct developers who modified specific files.
#     """
#     repo = Repo(repo_path)
#     author_stats = {}

#     for commit in repo.iter_commits():
#         author_name = commit.author.name
#         if author_name not in author_stats:
#             author_stats[author_name] = {"commit_count": 0, "files": set()}
#         author_stats[author_name]["commit_count"] += 1

#         for file in commit.stats.files:
#             author_stats[author_name]["files"].add(file)

#     return author_stats


# # File-Level Metric Calculations
# def calculate_file_metrics(modified_file):
#     """
#     Calculate file-level metrics.
#     Metrics:
#         - OWN: Percentage of lines contributed by the highest contributor in a file.
#         - NSCTR: Number of subsystems (directories) that a file belongs to.
#     """
#     old_path = modified_file.old_path if modified_file.old_path else ""
#     new_path = modified_file.new_path if modified_file.new_path else old_path
#     added_lines = modified_file.added_lines
#     deleted_lines = modified_file.deleted_lines

#     return {
#         "old_path": old_path,
#         "new_path": new_path,
#         "added_lines": added_lines,
#         "deleted_lines": deleted_lines,
#     }


# # Class-Level Metric Calculations using CK
# def calculate_class_metrics(repo_path):
#     """
#     Calculate class-level metrics for Java files using CK.jar.
#     Metrics:
#         - CBO: Coupling Between Object Classes.
#         - WMC: Weighted Methods Per Class.
#         - LCOM: Lack of Cohesion Methods.
#     """
#     os.system(f"java -jar ck.jar {repo_path} False 0 False > ck_output.txt")

#     ck_metrics = []
#     with open("ck_output.txt", "r") as ck_file:
#         for line in ck_file.readlines():
#             metric_data = line.strip().split()
#             if len(metric_data) == 13:
#                 file_path, cbo, wmc, lcom = (
#                     metric_data[0],
#                     metric_data[4],
#                     metric_data[5],
#                     metric_data[6],
#                 )
#                 ck_metrics.append(
#                     {"file_path": file_path, "CBO": cbo, "WMC": wmc, "LCOM": lcom}
#                 )

#     return ck_metrics


# # Metric calculation for each commit
# def calculate_all_metrics(repo_path, output_file):
#     # DataFrame to store all metrics
#     metrics_df = pd.DataFrame(
#         columns=[
#             "commit_hash",
#             "author",
#             "date",
#             "additions",
#             "deletions",
#             "num_files",
#             "class_metrics",
#         ]
#     )

#     # Traverse each commit in the repository
#     for commit in Repository(repo_path).traverse_commits():
#         commit_metrics = calculate_commit_metrics(commit)
#         file_metrics = [
#             calculate_file_metrics(mod_file) for mod_file in commit.modified_files
#         ]
#         class_metrics = calculate_class_metrics(repo_path)

#         # Append the data to the DataFrame
#         metrics_df = metrics_df.append(
#             {
#                 **commit_metrics,  # Unpack commit metrics
#                 "file_metrics": file_metrics,
#                 "class_metrics": class_metrics,
#             },
#             ignore_index=True,
#         )

#     # Save metrics to a CSV file
#     metrics_df.to_csv(output_file, index=False)
#     print(f"Metrics saved to {output_file}")


# # Visualization of metric evolution
# def plot_metric_evolution(metrics_csv):
#     df = pd.read_csv(metrics_csv)

#     # Plot additions, deletions, and number of files modified over time
#     plt.figure(figsize=(10, 6))
#     sns.lineplot(x="date", y="additions", data=df, label="Additions")
#     sns.lineplot(x="date", y="deletions", data=df, label="Deletions")
#     sns.lineplot(x="date", y="num_files", data=df, label="Modified Files")

#     plt.title("Code Changes Over Time")
#     plt.xlabel("Date")
#     plt.ylabel("Lines of Code / Files Modified")
#     plt.legend()
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()


# # Main function to run metric calculations for the given repo
# if __name__ == "__main__":
#     repo_path = "path/to/your/repo"
#     output_file = "metrics_output.csv"

#     print(f"Processing repository: {repo_path}")
#     calculate_all_metrics(repo_path, output_file)
#     plot_metric_evolution(output_file)